## Afternoon practical day 4

Welcome to the final practical of today. Here you'll be working with sequence data, first getting to the point that you have the sequences all aligned, and then using the distances you can only really calculate then to get a clustering of the sequences. As always, first run the two cells below. 



In [ ]:
#run this cell to set things up
import ipywidgets as widgets, numpy as np, pandas as pd
from numpy.random import default_rng
%matplotlib inline
import matplotlib.pyplot as plt
import math
import seaborn as sns
from IPython.display import display, Markdown
from scipy.optimize import fmin_bfgs, fmin_cg, fmin
import sklearn
import itertools
from Bio import SeqIO

In [ ]:
# important functions

def calcEucliDist(vectorOne, vectorTwo):
    return np.linalg.norm(vectorOne-vectorTwo, axis = 1)

def calcAbsDist(vectorOne, vectorTwo):
    #using linalg.norm:
    return np.linalg.norm(vectorOne-vectorTwo, ord = 1, axis = 1)

def makeKMeanClusters(X, k, funName = "calcEucliDist", maxIter = 50, nClusteringsToPerform = 20):
    if k <= 0:
        print("K must be greater than 0!")
        return None
    if k > len(X):
        print("K cannot be larger than the # of samples in your data!")
        return None
    if maxIter <= 0:
        print("Cannot have negative or 0 iterations!")
        return None
    
    resultToReturn = [None, None, None, None]
    bestDistortion = np.Inf
    
    for clusteringIndex in range(0, nClusteringsToPerform):
        initialCentroids   = X[np.random.choice(X.shape[0], k, replace=False), :]
        if len(initialCentroids) != k:
            print("Centroids lost!")
        centroids          = initialCentroids
        threeLastCentroids = []
        #print(centroids)
        for i in range(0, maxIter):

            threeLastCentroids.append(np.round(centroids, 4))
            distancesToCentroids = np.vstack([globals()[funName](centroids, datapoint) for datapoint in X])
            closestCentroid      = np.where(distancesToCentroids == np.amin(distancesToCentroids,
                                                                            axis = 1)[:, np.newaxis])[1]
            centroids            = np.vstack([np.mean(X[np.where(closestCentroid == clusterNum)],
                                                      axis = 0) for clusterNum in np.unique(closestCentroid)])

            if i >2:
                threeLastCentroids.pop(0)
                if np.array_equal(threeLastCentroids[-1],threeLastCentroids[-2]) and np.array_equal(threeLastCentroids[-2], threeLastCentroids[-3]):
                    print("No changes in cluster centroids detected in last 3 iterations. Finished at iteration " + str(i+1) + ".")
                    break
        
        # new code
        squareDistancesPerPoint = []
        for index, centroid in enumerate(closestCentroid):
            squareDistancesPerPoint.append(np.square(centroids[centroid, :] - X[index, :]))
        distortion = 1/len(X) * np.sum(np.array(squareDistancesPerPoint))
        
        if distortion < bestDistortion:
            bestDistortion = distortion
            resultToReturn = [centroids, closestCentroid, initialCentroids, bestDistortion]
                
    return resultToReturn

def hierarCluster(X, distanceFunc = "calcEucliDist", linkageMethod = "average", displayDistMatrix = False):
    
    if linkageMethod not in ["average", "complete", "single"]:
        print("Error, please input a valid linkage method!")
        return None
    if distanceFunc  not in globals().keys():
        print("Error, please input a valid distance function name!")
    
    # make an empty distance matrix
    distanceMatrix = np.zeros(shape = (len(X), len(X)))
    distanceMatrix.fill(np.nan)
    # make a list with the indices of every data point. This is the list of clusters, where you start
    # with every point in a cluster and then start merging them.
    initialList = [[index] for index, _ in enumerate(X)]
    clusterList = initialList.copy()
    clusteringOverIterations = []
    clusteringOverIterations.append(initialList)
    # also make an empty list that saves which cluster indices were merged for every iteration
    clusterIndicesMergedList = []
    for rowIndex, row in enumerate(distanceMatrix):
        for colIndex, cellValue in enumerate(row):
            # distance from yourself to yourself is 0, don't calculate!
            if colIndex == rowIndex:
                continue
            # in the first loop, you calculate distance from 1 to 2.
            # in the second loop, you don't want to calculate distance from 2 to 1 again. This safeguards against that.
            if colIndex < rowIndex:
                continue

            distanceMatrix[rowIndex, colIndex] = globals()[distanceFunc](X[rowIndex,:][np.newaxis,  :],
                                                                             X[colIndex, :][np.newaxis, :])
    if displayDistMatrix:
        display(pd.DataFrame(distanceMatrix))

    # We continue clustering until everything is in one giant cluster. Thats len(X)-1 clustering steps.
    for i in range(0, len(X)-1):
        # we start with no idea of which two clusters we need to cluster
        lowestDistDatapoints = None
        # since we haven't calculated any distance, our current distance is infinite
        distToCluster = np.Inf
        # clusterList initially looks like [[0], [1], ... [99]].
        # itertools.combinations makes that into [([0], [1]), ([0], [2]), ([0], [3]) ... ([1], [2]), ([1], [3])... (98, 99)]
        # so you get all possible combinations of clusters that you could cluster together
        for combo in itertools.combinations(clusterList, 2):

            distance = 0
            distanceSingleLink = np.Inf # need this because for single linkage you want lowest distance to be selected
                                        # so need to have the starting distance always be lower.
            # make all combinations of data points in the first cluster and data points in the second cluster
            # so if the current combo = ([0, 12, 15], [3, 2]), this results in:
            # [[0, 3], [0, 2], [12, 3], [12, 2], [15, 3], [15,2]]: these are all the points that we need to get
            # the distances for (and average for average linkage)
            toIterate = [j for i in [list(zip([elem] * len(combo[1]), combo[1] )) for elem in combo[0]] for j in i]
            for indicesTwoDatapoints in toIterate:
                #sort the indices. Our matrix has only the distance between 1 and 2, not between 2 and 1.
                #this turns [12, 2] from above into [2, 12], etc.
                indicesTwoDatapoints = sorted(indicesTwoDatapoints)

                # keep a running total of all distances between the points in the two clusters
                if linkageMethod == "average":
                    distance += distanceMatrix[indicesTwoDatapoints[0], indicesTwoDatapoints[1]]
                if linkageMethod == "complete":
                    # for a cluster, if the distance between two points is larger than the current largest distance
                    # between points in a cluster, that is the new cluster distance.
                    if distanceMatrix[indicesTwoDatapoints[0], indicesTwoDatapoints[1]] > distance:
                        distance = distanceMatrix[indicesTwoDatapoints[0], indicesTwoDatapoints[1]]
                if linkageMethod == "single":
                    if distanceMatrix[indicesTwoDatapoints[0], indicesTwoDatapoints[1]] < distanceSingleLink:
                        distanceSingleLink = distanceMatrix[indicesTwoDatapoints[0], indicesTwoDatapoints[1]]

            if linkageMethod == "average":
                totalAvgDistance = distance/(len(combo[0]) * len(combo[1]))

            # if distance between these clusters is less than the lowest distance we have seen so far,
            #set these clusters as the ones to cluster. 
                if totalAvgDistance < distToCluster:
                    distToCluster       = totalAvgDistance
                    dataPointsToCluster = combo

            if linkageMethod == "complete":
                if distance < distToCluster:
                    distToCluster       = distance
                    dataPointsToCluster = combo

            if linkageMethod == "single":
                if distanceSingleLink < distToCluster:
                    distToCluster       = distanceSingleLink
                    dataPointsToCluster = combo

        #make a new list of clusters
        clusterIndicesMergedList.append(dataPointsToCluster)
        clusterList = clusterList.copy()
        for index, elem in enumerate(clusterList):
            # merge the second cluster into the first cluster
            if elem == dataPointsToCluster[0]:
                clusterList[index] = clusterList[index] + dataPointsToCluster[1]
                #clusterList2[index] = sorted(clusterList[index])
            # remove the separate second cluster (it's now been merged to the first one)    
            if elem == dataPointsToCluster[1]:
                clusterList.pop(index)
        # Finally, save all clusters, from the very beginning (all separate clusters) until the very end (all in one cluster) in one list by appending to that the current clusters      
        clusteringOverIterations.append(clusterList)
        
        #addition to make a list of lists of everything:
        
    
    return [clusteringOverIterations, pd.DataFrame(distanceMatrix), clusterIndicesMergedList]


def drawHierarchicalClustering(hierarClusterOutcome, figsize = (25,8), title = "Plot", labels = None):
    clusterListX       = hierarClusterOutcome[0]
    clusteredPerStepX  = hierarClusterOutcome[2]
    xLabels            = np.array(list(itertools.chain(*clusterListX[-1])))
    
    fig, ax = plt.subplots(figsize = figsize)
    ax.set_xticks(range(0, len(xLabels)))
    if not labels is None:
        labels = np.array(labels)
        if len(labels) == len(xLabels):
            labels = labels[xLabels]
            ax.set_xticklabels(labels, rotation = 90)      
        else:
            print("Labels supplied should be of same length as the amount of data points!")
            return None
    else:   
        ax.set_xticklabels(xLabels)
    ax.margins(y=0)

    heightPerDataPointPreviousStep = np.array([0] * len(xLabels))
    for i, clusterStep in enumerate(clusteredPerStepX):
        pos1Positions = np.array([np.where(xLabels == elem)[0] for elem in clusterStep[0]])
        pos1Avg       = np.mean(pos1Positions)
        #pos1Start     = np.min(pos1Positions)
        #pos1End       = np.max(pos1Positions)
        pos1ClustSize = len(pos1Positions)
        pos2Positions = np.array([np.where(xLabels == elem)[0] for elem in clusterStep[1]])
        pos2Avg       = np.mean(pos2Positions)
        #pos2Start     = np.min(pos2Positions)
        #pos2End       = np.max(pos2Positions)
        pos2ClustSize = len(pos2Positions)



        heightEnd   = max(pos1ClustSize, pos2ClustSize)
        ax.plot([pos1Avg, pos1Avg], [heightPerDataPointPreviousStep[pos1Positions[0][0]],heightEnd], color = "black")
        ax.plot([pos2Avg, pos2Avg], [heightPerDataPointPreviousStep[pos2Positions[0][0]],heightEnd], color = "black")
        ax.plot([pos1Avg, pos2Avg], [heightEnd,heightEnd], color = "black")

        heightPerDataPointPreviousStep[np.ravel(pos1Positions)] += heightEnd - heightPerDataPointPreviousStep[pos1Positions[0][0]]
        heightPerDataPointPreviousStep[np.ravel(pos2Positions)] += heightEnd - heightPerDataPointPreviousStep[pos2Positions[0][0]]

    ax.set_ylim(0, max(heightPerDataPointPreviousStep)+1)
    fig.suptitle(title)

    plt.show(fig)

## Implementing Needleman-Wunsch for pairwise alignment

Before we can even think about clustering sequence data, we need a way to construct multiple sequence alignments. As you've been told, an often-used method (though it cannot guarantee the best possible MSA) is progressive tree alignment. There, you first make a hierarchical tree of the sequences based on some measure of how alike they are, and then align the most alike sequences, then align the next most alike to those two, etc. etc. until you've aligned all. In that way, you deal with the issue that with, say, 100 sequences that you all want to align together, you have so many possibilities that it's impossible to do, and you probably have many as good options (so which one, then, to pick?).

So you want to cluster sequence data. Well, you need this pairwise measure of similarity (or its inverse, distance). For that, the classic option is Needleman-Wunsch, a dynamic programming algorithm that breaks the problem of finding the global optimal alignment into the subproblem of finding the best subalignments possible, then tracing back from the maximum score to the beginning. We're going to be implementing this algorithm ourselves. Doubtless you have seen this once or twice, but you may need a refresher. Take your pick:

* [Video tutorial](https://www.youtube.com/watch?v=LhpGz5--isw)
* [Wikipedia](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm)

Let's implement it. First as a set of commands, then in a function, so we can use pairwise alignment to make a guide tree. We'll go in parts. The first part is making the table of scores. To do this:
* Initialise an array of zeros of size len(sequence1)+1 by len(sequence2)+1. Fill it with `np.nan`.
* Using the gap, match, and mismatch penalties below, fill in the table. To do that:
    * The leftmost column and top row are just filled by adding gap penalties from top to bottom and left to right, respectively
    * The most top left cell should contain 0.
    * The other cells are filled with the maximum score obtained by either:
        1. Stepping from the cell above (opening or extending a gap, i.e. adding the gap penalty to the score of the cell above)
        2. Stepping from the cell to the left (opening or extending a gap, i.e. adding the gap penalty to the score of the cell to the left)
        3. Stepping from the cell to the top left (i.e. diagonally, aligning the two residues, thereby either adding the match score or the mismatch score). <br>
      **Relevant Wikipedia screencap:** ![NW](SnippetNW.PNG)
* Make sure to use the scoring matrix for substitutions (so that we can, if we want, give another scoring matrix!)
* Your final output should be the matrix with scores per position, as described in the video and Wikipedia. We'll do the actual aligning below.

Hints:
* If all goes well, your output score matrix should look something like this (although it will be in Numpy array format rather than this pretty-printed pd.DataFrame format): ![outcome](NWScoreMatrixOutputForTestSequences.PNG) Note that it's perfectly valid if you have the sequences swapped (so seqTwo along the rows and seqOne along the columns).
* You can loop over a matrix/array by doing something like: <br>
`for rowIndex, rowValues in enumerate(array):` <br>
`    for colIndex, colValue in enumerate(rowValues:)` <br>
`        #do stuff` (do add the appropriate tabs)
* You add one row and one column to accomodate the gap penalties along the left and top. This means that if you want to add the mismatch or match for a certain combination of bases, you need to index with something like `seqOne[rowIndex-1]`.
* You can use `baseDict` to make your life easier `(baseDict["C"], baseDict["G"])` will give you the indices in the score matrix for this mismatch between a C and a G. So `scoreMatrix[baseDict["C"], baseDict["G"]]` will give you the value you want if there's a C in sequence one and a G in sequence two that you possibly want to mismatch.


In [ ]:
seqOne        = 'ATGCTTCG'
seqTwo        = 'ATGGCTGCCCC'
matchScore    = 1
mismatchScore = -1
gapScore      = -1

bases = ["A", "T", "G", "C"]
baseDict = dict(zip(bases, [0, 1, 2, 3]))
scoreMatrix = np.zeros(shape = (len(bases), len(bases)))
scoreMatrix[:] = mismatchScore
np.fill_diagonal(scoreMatrix, matchScore)


printTable = pd.DataFrame(scoreMatrix) ; printTable.columns = bases ; printTable.set_index(pd.Index(bases), inplace= True)
print("Substitution matrix for our simple case:")
display(printTable)


# your answer


## Adding the steps taken and constructing the sequence alignment

There's only two ingredients missing to get our alignment working:
1. We need to keep a record when we select the max value for our score table (either by moving from the left, top-left, or top) where we came from. This record is non-unique (we could get the same score for a gap as a mismatch, say).
2. Finally, we need to use this record to step back from the bottom right to the top left, going through the directions we saved. In real life, you'd want to report the alternative alignments possible. Here, we will just make one (remember, they are all as good as each other).

To do this:
* Copy your code from above into the cell below.
* Make an extra matrix called `stepMatrix` like so: `np.zeros_like(YourNameForTheScoreMatrixAbove).tolist()` (the tolist is because a numpy array only allows one value per cell. But we might want to store \['left', 'topleft'\] in one cell because we have two options).
* Your code already determined what the maximum score was. All you need to add is to check which entries gave that top score (an alignment/mismatch step (from topleft) or a gap step (from the left or from the top)) and add that as a value in `stepMatrix`. So you'd do something like `stepMatrix[rowIndex][colIndex] = ['left', 'topleft']`, if for a certain step both the step from the left cell and the top-left cell yield the same, maximum, score.

When that's done, you can move on to making the alignment. There's many slighly different ways to go about it, of course, but here's one idea:
* Start a rowIndex at `len(seqOne)`; start a colIndex at `len(seqTwo)`.
* Start two empty strings, `seqOneAligned` and `seqTwoAligned`.
* While both indices are not yet 0 (i.e. you are not yet at the top left):
    * Select the first entry from the stepMatrix at that position (there could be more, but for now we just make one of the optimal alignments).
    * If that entry is 'left', add "-" to `seqTwoAligned` and the letter at `seqOne[rowIndex-1]` to `seqOneAligned`, and subtract one from rowIndex.
    * If that entry is 'top', do the opposite (gap to seq one, letter to seq two, subtract one from colIndex)
    * If that entry is 'topleft', add both corresponding letters, and subtract one from both rowIndex and colIndex
* When done, reverse the two sequences (they're now back-to-front) and voila: aligned sequences!


In [ ]:
# your answer here:





## Making a function out of this.

Now, functionalise what you've been implementing so far. Call the function `alignNW()`, give it 4 arguments `(seqOne, seqTwo, substitutionMatrix = scoreMatrix, substitutionDict = baseDict)`. It should return a dictionary with three entries: "similarityScore", "seqOneAligned", and "seqTwoAligned". similarityScore is just the bottom right value in the score matrix: it is the measure of how similar the two sequences are based on their alignment. The other two speak for themselves. As a sanity check, `alignNW('CCC', 'CCC')` should return a score of 3, while `alignNW('ATG', 'GTA')` should return -1.

Note: don't print anything anymore if you did before. We're going to use this function a lot of times!

In [ ]:
# set-up
matchScore    = 1
mismatchScore = -1
gapScore      = -1

bases = ["A", "T", "G", "C"]
baseDict = dict(zip(bases, [0, 1, 2, 3]))
scoreMatrix = np.zeros(shape = (len(bases), len(bases)))
scoreMatrix[:] = mismatchScore
np.fill_diagonal(scoreMatrix, matchScore)

#define alignNW




## Data for constructing a guide tree

The reason I'm suddenly making you implement pairwise alignment is that if I ask you to cluster sequences, you won't be able to do it just like that. The reason for that is that, while in the morning practicals we had data points with features 1 to _n_ for each of them, for sequences you don't know which letter is which feature. Maybe the 3rd letter in sequence 43 actually corresponds to the 7th letter in sequence 22, because of deletions or duplications. To know what corresponds to what, we need a multiple sequence alignment.

That sounds fine and dandy, but as you've heard, making a multiple sequence alignment of a 100 sequences together is basically impossible: there are so many options it's infeasible to calculate. So we compromise, getting a very good (but not per se optimal) MSA by doing _progressive tree alignment_: we get some notion of pairwise distances between sequences (either via pairwise alignment or k-mer or k-tuple methods), and then align the two closest sequences, then the one that's closest to either of those two, etc.

Now that we have Needleman-Wunsch (NW), I'd like you to construct a guide tree. Note that the final score in the lower right of the NWscore matrix is the similarity score for two sequences, so the inverse of their distance. Below I give you 50 sequences of ribosomal 16S subunits, which I randomly selected from the RDP database unaligned data file that can be found [here](http://rdp.cme.msu.edu/misc/resources.jsp). We will be aligning the first 80 bases of 25 of those: I thought I could let you align the 1225 unique combinations of 50 by 50 full sequences, but that would take _ages_ with our naive implementation (which shows you how dire the problem of computational expense can be)!

We'll be using the [DNAFULL scoring matrix](https://rosalind.info/glossary/dnafull/), which includes all the iupac symbols for nucleotide sequences (like N for 'any nucleotide at this position, we don't know which, only that there is a nucleotide here'). 

* Make a matrix of zeros of size 25\*25. 
* Align all unique combinations of the 25 16S rRNA subsequences in `sequences` using the indices in `combinations`. Save them in a similarity matrix.
* Make the similarity matrix into a distance matrix by: 
    * Adding (-(minimum score in matrix) +1) to all entries (there could be negative scores)
    * Doing 1/score (so very high similarity of 100 becomes 1/100 = small distance, while 1/1 (completely dissimilar) is just 1.


In [ ]:
# advanced scoring matrix:

scoreMatDNAFULL = np.array([ [ 5,  -4,  -4,  -4,  -4,   1,   1,  -4,  -4,   1,  -4,  -1,  -1,  -1,  -2],
            [-4,   5,  -4,  -4,  -4,   1,  -4,   1,   1,  -4,  -1,  -4,  -1,  -1,  -2],
            [-4, -4,   5,  -4,   1,  -4,   1,  -4,   1,  -4,  -1,  -1,  -4,  -1,  -2],
            [-4,  -4,  -4,   5,   1,  -4,  -4,   1,  -4,   1,  -1,  -1,  -1,  -4,  -2],
[-4,  -4,   1,   1,  -1,  -4,  -2,  -2,  -2,  -2,  -1,  -1,  -3,  -3,  -1],
[1,   1,  -4,  -4,  -4,  -1,  -2,  -2,  -2,  -2,  -3,  -3,  -1,  -1,  -1],
[1,  -4,   1,  -4,  -2,  -2,  -1,  -4,  -2,  -2,  -3,  -1,  -3,  -1,  -1],
[-4,   1,  -4,   1,  -2,  -2,  -4,  -1,  -2,  -2,  -1,  -3,  -1,  -3,  -1],
[-4,   1,   1,  -4,  -2,  -2,  -2,  -2,  -1,  -4,  -1,  -3,  -3,  -1,  -1],
[1,  -4,  -4,   1,  -2,  -2,  -2,  -2,  -4,  -1,  -3,  -1,  -1,  -3,  -1],
[-4,  -1,  -1,  -1,  -1,  -3,  -3,  -1,  -1,  -3,  -1,  -2, -2,  -2,  -1],
[-1,  -4,  -1,  -1,  -1,  -3,  -1,  -3,  -3,  -1,  -2,  -1,  -2,  -2,  -1],
[-1,  -1,  -4,  -1,  -3,  -1,  -3,  -1,  -3,  -1,  -2,  -2,  -1,  -2,  -1], 
[-1,  -1,  -1,  -4,  -3,  -1,  -1,  -3,  -1,  -3,  -2,  -2,  -2,  -1,  -1],
[-2,  -2,  -2,  -2,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1]])

basesDNAFULL = ["A", "T", "G", "C", "S", "W", "R", "Y", "K", "M", "B", "V", "H", "D", "N"]
dictDNAFULL = dict(zip(basesDNAFULL, range(0, len(basesDNAFULL))))


# reading data
file_in               = "firstFifty16SSequences.fasta"
sequenceDict          = SeqIO.to_dict(SeqIO.parse(open(file_in, mode='r'), 'fasta'))
finalBasePosToInclude = 80
nSequencesToUse       = 25
sequences = [str(elem.seq).upper()[0:finalBasePosToInclude] for index, elem in enumerate(list(sequenceDict.values())) if index <nSequencesToUse]

# again, we don't want to align seq 1 with seq 2 and seq 2 with seq 1, they're the same
# combinations gets the unique combinations for us.
combinations = list(itertools.combinations(range(0, len(sequences)), 2))
print("Alignments to perform: " + str(len(combinations)))

#up to you!


## Making the guide tree

Now all we need to do is construct a hierarchical tree out of the sequence distances and we'll know how to do progressive tree alignment. For that, we need to change our hierarCluster function, allowing it to skip calculating distances and instead make a clustering on pre-defined distances. To do this:

* Add an argument `distMatrix = None` to the function below
* If `distMatrix` is not None, it should use the pre-calculated distance matrix instead and skip the whole distance calculation step. 
* Run this updated function, with X just being `sequences` from above.
* Use `drawHierarchicalClustering` to see the results. 
* Check that the first two sequences that are clustered correspond to the minimum of the distance matrix you calculated in the previous cell. Notice anything strange?

Hint:
* For the last point you can use `np.where()` and `np.min()`.

In [ ]:
# Your answer here



## Calculating similarity scores for guide trees more rapidly

This approach of pairwise alignment also rapidly becomes infeasible, as evidenced by the fact that I only let you align a few sequences, and of those, only part of the sequence, because otherwise it would take too long (with our simple implementation). Nevermind if you have 10.000 or 100.000 sequences to align. Then, different methods, like k-mers or k-tuples are used. We'll implement the k-mer method of distance calculation here for the sequences above. 

K-mers are subsequences of length k in a sequence. So a 1-mer in a nucleotide sequence are just the amount of A, T, G, and C in that sequence. But for 2-mers, you get the amount of AA, AT, TA, TT, GC, CG, GG, CC, CT, TC, TG, GT, AC, CA, AG, GA in it. See this image: <br> ![Kmers](Kmers.PNG)

Note that the k-mers overlap. You might already see the advantage: k-mers are quick to calculate, and they give you a set number of features for each sequence. You can then easily calculate distances based on these features and cluster hierarchically: bam, you have your guide tree! Read more [here](https://en.wikipedia.org/wiki/K-mer).

Let's make a guide tree for the sequences from above (although now all of them, and with their full length) using 2- and 3-mers. The code below is your starting point. Up to you to calculate the k-mer counts for each sequence and do the hierarchical clustering (with Euclidean distance calculation). To do this:

* Make an empty list to hold the features for each sequence.
* Go over each sequence, for each:
    * Make an empty feature vector with 80 spots for every k-mer
    * Glide over the sequence with a sliding window, incrementing the correct spot for the 2-mer and 3-mer you read at each position 
    * Append the features for this sequence to the list you made in the beginning.
* Finally, use `np.vstack()` to stack all these features, you should end up with a 50 by 80 feature matrix X for hierarchical clustering.
* After this, do the hierarchical clustering using average linkage and Euclidean distance, visualise it, and try to see how much it differs from the one with pairwise alignment.

Hints:
* You can increment the 3-mer and 2-mer at the same time, provided you realise that the 3-mer will give an error once the sliding window is at the very end of the string. So you should check that `currentPos+3 is not > len(currentSeq)`
* In the answers, I have done this procedure also for the exact same subsequences as we used above. Look there if you want to.

In [ ]:
allSequences = [str(elem.seq).upper() for index, elem in enumerate(list(sequenceDict.values()))]

twoMers   = list(itertools.product("ATGC", repeat=2))
twoMers   = ["".join(elem) for elem in twoMers]
threeMers = ["".join(elem) for elem in list(itertools.product("ATGC", repeat=3))]
mersWeUse = np.array(twoMers + threeMers)

print("Each sequence will now have " + str(len(mersWeUse)) + " features.")

# your turn:






## Going from the guide tree to the multiple sequence alignment

Now that we have our guide tree (either based on k-mers or pairwise distances) we can perform progressive sequence alignment. What I haven't gone into details on is _how_ we align sequence C to already pairwise-aligned sequence A and B. Or two aligned sets of sequences (A, B) and (C, D) to each other. Many complex scoring schemes could be thought up, but the simplest is just to treat every column of the two pairwise alignments as one 'letter' and align the sequences as you normally would using something like Needleman-Wunsch.

To illustrate:
If we have the two protein (sub)sequences: <br>
AIKA <br>
AL-A

and 

ALA <br>
VLA

We could do something like on the following image, where the score for accepting something is the sum (or average) of aligning A with A, A with A, and V with A and V with A for the first position: ![image](MSA.PNG)

Modifying our `alignNW()` function for this is possible, though it would take quite some work and might cause some headaches. Instead, I think you now understand the principles of progressive tree alignment well enough to use something like Clustal Omega. Up to you to:

* Go [here](https://www.ebi.ac.uk/Tools/msa/clustalo/) and use Clustal Omega to align the 50 16s RNA sequences.
* Compare the guide tree to the full k-mer guide tree. If you didn't do this before, you can set `labels = list(sequenceDict.keys())` when plotting the k-mer guide tree so that the leaves are labeled with the sequence ID, just like in Clustal Omega. Do the guide trees look alike or highly dissimilar?
* Download the MSA file. With that in hand, we can _finally_ cluster the sequences based on some distances between them, resulting in one of the most well-known clustering plots in biology: a phylogenetic tree.

In [ ]:
# your answer



## Using the multiple-sequence alignment to get a phylogenetic tree

Okay then, nearly there. We've clustered the sequences based on some characteristics (pairwise alignment scores or k-mer scores, or k-tuple score) to get a guide tree. Then, we've made a multiple-sequence alignment (MSA) _along_ that guide tree, with the logic that it's easiest to align the most alike sequences first (you'll make the least errors there) and progressively keep adding sequences to that so you'll finally get a quite good (though not optimal) MSA. Now, finally, with the MSA in hand, we have separate features (here, if yours is the same as mine, 1558 features, i.e. 1558 positions in the MSA) on which to cluster our sequences. The only thing that's left to do is to construct a distance matrix, and then cluster.

To construct the distance matrix, we'll just take a simple criterion: the distance from sequence A to sequence B increases by 1 if in position _n_ they do not have the same character (be that A, T, G, C, N, -, etc.). In reality, of course, here too we could use more complex scores, where different substitutions that are more or less likely change the distance more or less. We won't bother with that and keep it simple. Now, it's up to you to make this distance matrix and construct the phylogenetic tree. To do that:

* Use `itertools.combinations` to get every unique combination of 2 sequence indices.
* For each combination of sequences, slide over them letter by letter (or gap by gap). If they are the same: do nothing. If they are different: add 1 to the distance for this pair. Note: you can select the first sequence in the alignment as a string like so: `str(align[0, :].seq)`.
* Put the distances in a matrix.
* Use this distance matrix to make a phylogenetic tree, use average linkage. 
* Finally, compare that phylogenetic tree to the one Clustal Omega gave you.

Hints:
* To get all the sequence names for use as labels, use `[elem.id for elem in align]`.

In [ ]:
from Bio import AlignIO
yourAlignFile = ''
#align = AlignIO.read(yourAlignFile, "clustal")


# your answer



## What I want you to remember here:
* How Needleman-Wunsch works and finds globally optimal pairwise alignments
* How the most well-known application of (hierarchical) clustering in biology, making phylogenetic trees, requires a twist or certain circularity because you don't know which letters in a sequence correspond to which letters in another sequence.
* How this forces you to do an initial clustering on some property (pairwise alignment distances if possible, otherwise k-mer methods or even mBed methods) to make a _guide tree_ that you can then align along.
* That with the guide tree in hand, you can finally calculate the evolutionary distances that you want, and cluster hierarchically based on those to get your phylogenetic tree.
* That the model of evolution used by alignment algorithms is necessarily an indel-model. We know that evolution actually happens via many duplications, sometimes even whole-genome duplications, but alignment programs like Clustal Omega will never tell you directly 'oh, this is a duplication'. Instead you might get 2 equally good global alignments and have to deduce that this happened yourself. In other words: _you still need to think about what the clustering criteria you use actually do for the type of evolution you can measure well in a phylogenetic tree_. Of course there's a lot more nuance, but you can go to Berend for that (see below).


## The end

That concludes our school trip to clustering in phylogenetics. I hope you aren't nauseous from all the candy you kids are wont to consume on such trips. Please do note that this is only a _tiny_ piece of the vast difficulties in the field of phylogenetics, and that you should definitely check out the course taught by Prof. Dr. Berend Snel on the subject for biological background and methodical details.

## Survey
Go on, eat your [survey](https://docs.google.com/forms/d/e/1FAIpQLSfC8YzEjnv0b0iEy7Hgs4fbRMY8oH8XMkSeW3Fl97tmjKseBQ/viewform?usp=sf_link), it'll make you grow big and strong!

